Import Necessary Packages

In [1]:
import generate_clusters
import generate_plotly
import json
import matplotlib.pyplot as plt
import networkx as nx
import network_graph
import numpy as np
import pandas as pd
import plotly 
import plotly.plotly as py
from plotly.graph_objs import *
import pylab
import set_plotly_credentials as setply
from sklearn.cluster import KMeans

TabError: inconsistent use of tabs and spaces in indentation (generate_plotly.py, line 26)

In [2]:
setply.set_plotly_credentials('credentials.json')

In [8]:
!ls

PandasDateMatch.ipynb         Untitled.ipynb
README.md                     __pycache__
Speed Dating Data Key.doc     credentials.json
Speed Dating Data.csv         generate_clusters.py
Speed+Dating+Script+New.ipynb generate_plotly.py
UWCapstone.ipynb              network_graph.py
Untitled                      set_plotly_credentials.py


Read in Necessary Dataset

In [3]:
filepath = 'https://raw.githubusercontent.com/rmw647/not-that-into-you/master/Speed%20Dating%20Data.csv'
data = pd.read_csv(filepath, encoding='latin1')

Choose which variables you want to cluster on.

In [4]:
bio_info = data[['iid','pid','match','age','sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga','exphappy']]

In [5]:
bio_info = bio_info.dropna(how='any')

Only keep one line of background information data for each participant.

In [6]:
bio_info_subset = bio_info.drop_duplicates(subset='iid')

Select Out the Speed-Dating Matches for Links

In [7]:
matches = bio_info[['iid','pid','match']]
matches = matches[matches.match==1]
matches['pid'] = matches['pid'].astype(int)
matches = matches[matches.iid.isin(bio_info_subset.iid)&matches.pid.isin(bio_info_subset.iid)]
edges = list(zip(matches.iid, matches.pid))


Perform K-means clustering. Normalize and generate cluster labels.

In [9]:
cluster_columns = ['age','sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing',
                   'reading','tv','theater','movies','concerts','music','shopping','yoga','exphappy']

bio_info_subset = generate_clusters.generate_clusters(bio_info,cluster_columns,3,'cluster_assignment')

In [12]:
G, clusters = network_graph.network_graph(bio_info_subset.iid, edges, bio_info_subset.cluster_assignment)

In [496]:
generate_plotly(G)

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~1215Diana/0 or inside your plot.ly account where it is named 'networkx'


In [ ]:
new = pd.DataFrame.from_dict(cluster, orient='index').reset_index()
new.rename(columns={'index': 'iid', 0: 'cluster'}, inplace=True)
together=pd.merge(new,matches,left_on=new.iid,right_on=matches.iid,how='right')
together2=pd.merge(together,new,left_on=together.pid,right_on=new.iid,how='left')
together2=together2.groupby(['iid_x', 'iid']).last()
together2['difference']=together2.cluster_x-together2.cluster_y
together2['difference'].value_counts().plot( kind='bar')
plt.show()